<a href="https://colab.research.google.com/github/jexiao8211/CSDS312-FINAL-PROJECT/blob/main/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc -V && which nvcc

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
/usr/local/cuda/bin/nvcc


In [2]:
!nvidia-smi

Mon May  1 06:09:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.


In [4]:
!python3 -m pip install pynvml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!python3 rapidsai-csp-utils/colab/env-check.py

Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/env-check.py", line 28, in <module>
    if ('K80' not in gpu_name):
TypeError: a bytes-like object is required, not 'str'


In [6]:
!python rapidsai-csp-utils/colab/pip-install.py

Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 28, in <module>
    if ('K80' not in gpu_name):
TypeError: a bytes-like object is required, not 'str'


In [7]:
import cudf
cudf.__version__

'23.04.01'

In [8]:
import cuml
cuml.__version__

'23.04.01'

In [9]:
import cudf
import numpy as np
import pandas as pd
import pickle

from cuml.ensemble import RandomForestClassifier as curfc
from cuml.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier as skrfc
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [10]:
# Replace 'your_file.csv' with the name of your CSV file
file_path = 'final_data.csv'

# Read the CSV file using pandas
df = pd.read_csv(file_path)

# Drop unnamed columns
unnamed_columns = df.columns[df.columns.str.contains('^Unnamed')]
df = df.drop(columns=unnamed_columns)

df = df.dropna()

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18584 entries, 0 to 19560
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   home_matches_played          18584 non-null  int64  
 1   home_wins                    18584 non-null  int64  
 2   home_win_rate                18584 non-null  float64
 3   home_buildUpPlaySpeed        18584 non-null  float64
 4   home_buildUpPlayDribbling    18584 non-null  float64
 5   home_buildUpPlayPassing      18584 non-null  float64
 6   home_chanceCreationPassing   18584 non-null  float64
 7   home_chanceCreationCrossing  18584 non-null  float64
 8   home_chanceCreationShooting  18584 non-null  float64
 9   home_defencePressure         18584 non-null  float64
 10  home_defenceAggression       18584 non-null  float64
 11  home_defenceTeamWidth        18584 non-null  float64
 12  away_matches_played          18584 non-null  int64  
 13  away_wins       

In [11]:
# n_samples = 2*17
n_samples = 2**12
n_features = 399
n_info = 300
data_type = np.float32

In [66]:
predict_columns = ['home_win', 'draw', 'away_win']
feature_columns = [col for col in df.columns if col not in predict_columns]

X = df[feature_columns]
y = pd.DataFrame(np.argmax(df[predict_columns].values, axis=1))

### Local Host

In [67]:
X = cudf.DataFrame(X)
y = cudf.DataFrame(y)[0]

In [69]:
%%time
# X = pd.DataFrame(X.astype(data_type))
# # cuML Random Forest Classifier requires the labels to be integers
# y = pd.Series(y.astype(np.int32))

X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=42)

CPU times: user 16.7 ms, sys: 1.04 ms, total: 17.8 ms
Wall time: 19.2 ms


In [72]:
# Copy dataset from GPU memory to host memory.
# This is done to later compare CPU and GPU results.
X_train = X_cudf.to_pandas()
X_test = X_cudf_test.to_pandas()
y_train = y_cudf.to_pandas()
y_test = y_cudf_test.to_pandas()

### GPU

In [60]:
# %%time
# X_cudf_train = cudf.DataFrame.from_pandas(X_train)
# X_cudf_test = cudf.DataFrame.from_pandas(X_test)

# y_cudf_train = cudf.Series(y_train.values)

# Random Forest

### Fit Model

In [73]:
# fit sklearn RandomForest
%%time
sk_model = skrfc(n_estimators=40,
                 max_depth=16,
                 max_features=1.0,
                 random_state=10)

sk_model.fit(X_train, y_train)

CPU times: user 5.73 s, sys: 20.1 ms, total: 5.75 s
Wall time: 5.93 s


RandomForestClassifier(max_depth=16, max_features=1.0, n_estimators=40,
                       random_state=10)

In [74]:
# evaluate
%%time
sk_predict = sk_model.predict(X_test)
sk_acc = accuracy_score(y_test, sk_predict)

CPU times: user 57.5 ms, sys: 880 µs, total: 58.4 ms
Wall time: 60.5 ms


In [75]:
# cuML RandomForest
%%time
cuml_model = curfc(n_estimators=40,
                   max_depth=16,
                   max_features=1.0,
                   random_state=10)

cuml_model.fit(X_train, y_train)

/usr/local/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/usr/local/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


CPU times: user 297 ms, sys: 232 ms, total: 529 ms
Wall time: 535 ms


RandomForestClassifier()

In [76]:
# evaluate
%%time
fil_preds_orig = cuml_model.predict(X_cudf_test)

fil_acc_orig = accuracy_score(y_test.to_numpy(), fil_preds_orig)

CPU times: user 1.07 s, sys: 119 ms, total: 1.19 s
Wall time: 1.12 s


### Pickle cuML random forest classification model

In [77]:
filename = 'cuml_random_forest_model.sav'
# save the trained cuml model into a file
pickle.dump(cuml_model, open(filename, 'wb'))
# delete the previous model to ensure that there is no leakage of pointers.
# this is not strictly necessary but just included here for demo purposes.
del cuml_model
# load the previously saved cuml model from a file
pickled_cuml_model = pickle.load(open(filename, 'rb'))

In [78]:
# predict using the pickled model
%%time
pred_after_pickling = pickled_cuml_model.predict(X_cudf_test)

fil_acc_after_pickling = accuracy_score(y_test.to_numpy(), pred_after_pickling)

CPU times: user 256 ms, sys: 128 ms, total: 385 ms
Wall time: 394 ms


### Compare Results

In [79]:
print("CUML accuracy of the RF model before pickling: %s" % fil_acc_orig)
print("CUML accuracy of the RF model after pickling: %s" % fil_acc_after_pickling)

CUML accuracy of the RF model before pickling: 0.4784714877605438
CUML accuracy of the RF model after pickling: 0.4784714877605438


In [80]:
print("SKL accuracy: %s" % sk_acc)
print("CUML accuracy before pickling: %s" % fil_acc_orig)

SKL accuracy: 0.48493003845214844
CUML accuracy before pickling: 0.4784714877605438


# Linear Regression

In [81]:
import cudf
from cuml import make_regression, train_test_split
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression

In [83]:
%%time
X = cudf.DataFrame(X)
y = cudf.DataFrame(y)[0]

X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=42)

CPU times: user 20.1 ms, sys: 3.87 ms, total: 24 ms
Wall time: 52.8 ms


In [84]:
# Copy dataset from GPU memory to host memory.
# This is done to later compare CPU and GPU results.
X_train = X_cudf.to_pandas()
X_test = X_cudf_test.to_pandas()
y_train = y_cudf.to_pandas()
y_test = y_cudf_test.to_pandas()

## Scikit-learn Model

### Fit Model

In [85]:
%%time
ols_sk = skLinearRegression(fit_intercept=True,
                            n_jobs=-1)

ols_sk.fit(X_train, y_train)

CPU times: user 20.8 ms, sys: 5.96 ms, total: 26.7 ms
Wall time: 32.6 ms


LinearRegression(n_jobs=-1)

### Predict

In [86]:
%%time
predict_sk = ols_sk.predict(X_test)

CPU times: user 3.42 ms, sys: 3.01 ms, total: 6.44 ms
Wall time: 6.52 ms


### Evaluate

In [87]:
%%time
acc_sk_linreg = accuracy_score(y_test, predict_sk)

CPU times: user 3.03 ms, sys: 3 µs, total: 3.03 ms
Wall time: 3.04 ms


## cuML Model

### Fit Model

In [88]:
%%time
ols_cuml = cuLinearRegression(fit_intercept=True,
                              algorithm='eig')

ols_cuml.fit(X_cudf, y_cudf)

CPU times: user 216 ms, sys: 3.99 ms, total: 220 ms
Wall time: 219 ms


LinearRegression()

### Predict

In [89]:
%%time
predict_cuml = ols_cuml.predict(X_cudf_test)

CPU times: user 8.63 ms, sys: 44 µs, total: 8.68 ms
Wall time: 9.86 ms


### Evaluate

In [90]:
%%time
acc_origin_linreg = accuracy_score(y_cudf_test.to_numpy(), predict_cuml)

CPU times: user 4.18 ms, sys: 0 ns, total: 4.18 ms
Wall time: 4.24 ms


## Pickle Linear Regression Model

In [91]:
filename = 'cuml_linear_regression_model.sav'
# save the trained cuml model into a file
pickle.dump(ols_cuml, open(filename, 'wb'))
# delete the previous model to ensure that there is no leakage of pointers.
# this is not strictly necessary but just included here for demo purposes.
del ols_cuml
# load the previously saved cuml model from a file
pickled_cuml_model = pickle.load(open(filename, 'rb'))

In [92]:
# predict using the pickled model
%%time
pred_after_pickling = pickled_cuml_model.predict(X_cudf_test)

fil_acc_after_pickling = accuracy_score(y_test.to_numpy(), pred_after_pickling)

CPU times: user 10.1 ms, sys: 951 µs, total: 11 ms
Wall time: 15.3 ms


### Compare Results

In [93]:
print("CUML accuracy of the RF model before pickling: %s" % acc_origin_linreg)
print("CUML accuracy of the RF model after pickling: %s" % fil_acc_after_pickling)

CUML accuracy of the RF model before pickling: 0.47416576743125916
CUML accuracy of the RF model after pickling: 0.47416576743125916


In [94]:
print("SKL accuracy: %s" % acc_sk_linreg)
print("CUML accuracy before pickling: %s" % acc_origin_linreg)

SKL accuracy: 0.47416576743125916
CUML accuracy before pickling: 0.47416576743125916


# SVC

In [95]:
import cudf
import numpy as np
from cuml.metrics import accuracy_score
from cuml.svm import SVC as cumlSVC
from sklearn.svm import SVC as skSVC

## Scikit-learn Model

### Fit Model

In [ ]:
%%time
sk_svc = skSVC(kernel='poly', degree=2, gamma='auto', C=1)
sk_svc.fit(X_train, y_train)

### Predict

In [ ]:
%%time
predict_sk_svc = sk_svc.predict(X_test)
predict_sk_svc.tail()

### Evaluate

In [ ]:
%%time
acc_sk_svc = accuracy_score(y_test, predict_sk_svc)

## cuML Model

### Fit Model

In [ ]:
%%time
cuml_svc = cumlSVC(kernel='poly', degree=2, gamma='auto', C=1)
cuml_svc.fit(X_train, y_train)

### Predict

In [ ]:
%%time
predict_cuml_svc = cuml_svc.predict(X_test)
predict_cuml_svc.tail()

### Evaluate

In [ ]:
%%time
acc_origin_svc = accuracy_score(y_test, predict_cuml_svc)

## Pickle SVC Model

In [ ]:
filename = 'cuml_svc_model.sav'
# save the trained cuml model into a file
pickle.dump(cuml_svc, open(filename, 'wb'))
# delete the previous model to ensure that there is no leakage of pointers.
# this is not strictly necessary but just included here for demo purposes.
del cuml_svc
# load the previously saved cuml model from a file
pickled_cuml_model = pickle.load(open(filename, 'rb'))

In [ ]:
# predict using the pickled model
%%time
pred_after_pickling_svc = pickled_cuml_model.predict(X_test)

fil_acc_after_pickling_svc = accuracy_score(y_test.to_numpy(), pred_after_pickling_svc)

### Compare Results

In [ ]:
print("CUML accuracy of the RF model before pickling: %s" % acc_origin_svc)
print("CUML accuracy of the RF model after pickling: %s" % fil_acc_after_pickling_svc)

In [ ]:
print("SKL accuracy: %s" % acc_sk_svc)
print("CUML accuracy before pickling: %s" % acc_origin_svc)